# Week 7 Diversity Analysis, Rarefaction and Significance Tests

In this tutorial we will be looking into diversity analysis with preprocessed feature tables obtained from the Human Microbiome Project. For this we will be going through the following steps:         


[1. Download and import datasets](#sec1)                
[2. Alpha rarefaction](#sec2)         
[3. Diversity analysis](#sec3)         
[3.1 Alpha diversity](#sec3.1)            
[3.2 Beta diversity](#sec3.2)              

In [1]:
# importing all required packages & notebook extensions at the start of the notebook
import os
from qiime2 import Visualization

! jupyter serverextension enable --py qiime2 --sys-prefix

Enabling: qiime2.jupyter
- Writing config: /home/qiime2/miniconda/envs/qiime2-2021.8/etc/jupyter
    - Validating...
      qiime2.jupyter  OK


In [2]:
# assigning variables throughout the notebook

# location of this week's data and all the results produced by this notebook 
# - this should be a path relative to your working directory
data_dir = 'w7_data'

# Create a folder in current working directory to save downloaded files into:
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

<a id='sec1'></a>

# 1. Download and import datasets

## 1.1 Download the metadata

Metadata of the Human Microbiome Project has been processed and investigated in Exercises of Week 2. Here we are downloading this pre-processed file into the `w7_data` directory:

In [3]:
! wget -O $data_dir/metadata_proc.tsv https://polybox.ethz.ch/index.php/s/RSLMfTo0nU46vMN/download

--2021-11-01 15:24:48--  https://polybox.ethz.ch/index.php/s/RSLMfTo0nU46vMN/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350610 (342K) [application/octet-stream]
Saving to: ‘w7_data/metadata_proc.tsv’

w7_data/metadata_pr 100%[===================>] 342.39K  --.-KB/s    in 0.04s   

2021-11-01 15:24:48 (7.55 MB/s) - ‘w7_data/metadata_proc.tsv’ saved [350610/350610]



## 1.2 Download the feature table

The feature table is already demultiplexed, denoised with Deblur and filtered for adequate size. This Q2 artifact of type `FeatureTable[Frequency]` can be downloaded with:


In [4]:
! wget -O $data_dir/feature-table.qza https://polybox.ethz.ch/index.php/s/vAThxGGcKWmwoqI/download

--2021-11-01 15:24:49--  https://polybox.ethz.ch/index.php/s/vAThxGGcKWmwoqI/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1610071 (1.5M) [application/octet-stream]
Saving to: ‘w7_data/feature-table.qza’

w7_data/feature-tab 100%[===================>]   1.54M  9.46MB/s    in 0.2s    

2021-11-01 15:24:49 (9.46 MB/s) - ‘w7_data/feature-table.qza’ saved [1610071/1610071]



Create a summary of the read in feature table (takes ~2min to run) and inspect the created visualisation `w7_data/sum_table.qzv` inline with the Q2 notebook extension (as illustrated below) or in [QIIME2 View](https://view.qiime2.org/):

In [5]:
! qiime feature-table summarize \
  --i-table $data_dir/feature-table.qza \
  --m-sample-metadata-file $data_dir/metadata_proc.tsv \
  --o-visualization $data_dir/feature-table.qzv

Saved Visualization to: w7_data/feature-table.qzv


In [6]:
Visualization.load(f'{data_dir}/feature-table.qzv')

<visualization: Visualization uuid: 8ac5e9da-0fd9-4832-9fb5-168a0174a0c2>

## 1.3 Download a pre-created phylogenetic tree

As some diversity metrics incorporate evolutionary relatedness between sequences, we are downloading a pre-created phylogenetic tree for our dataset. The tree was created with a fragment insertion tree building method (`sepp` action of [q2-fragment-insertion](https://library.qiime2.org/plugins/q2-fragment-insertion/16/) plugin).

In [7]:
! wget -O $data_dir/insertion-tree.qza \
    https://polybox.ethz.ch/index.php/s/f7ltNX7sWQcDTcF/download

--2021-11-01 15:24:56--  https://polybox.ethz.ch/index.php/s/f7ltNX7sWQcDTcF/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3030509 (2.9M) [application/octet-stream]
Saving to: ‘w7_data/insertion-tree.qza’

w7_data/insertion-t 100%[===================>]   2.89M  11.2MB/s    in 0.3s    

2021-11-01 15:24:57 (11.2 MB/s) - ‘w7_data/insertion-tree.qza’ saved [3030509/3030509]



<a id='sec2'></a>

# 2. Alpha rarefaction

Having downloaded all the datasets we need in this exercise, we will now investigate the sampling depth of our sequences. As sampling depth is different between samples, we must normalize them prior to further analysing them. There is active research being performed on which type of normalisation method is best suited. (You will hear more about this in next week's lecture.)              
In this exercise we will normalise by sampling a random subset of sequences without replacement for each sample at a fixed sequencing depth ("threshold"). This approach is called "rarefying" and occurs in two steps: first, __sequences__ below the rarefaction threshold are filtered out of the feature table, then, the remaining __sequences__ are subsampled w/o replacement to get to the specified sequencing depth.             
To perform rarefaction, we first need to decide which rarefying threshold is best suited for our dataset. For this, we will analyse how sampling depth impacts within-sample diversity estimates (= **alpha diversity**) with the `alpha-rarefaction` action. This action generates interactive alpha rarefaction curves for sequencing depths between `min_depth` and `max_depth` and computes 10 (default) rarefied tables with corresponding alpha diversity metrics at each sampling depth step. 

* Before using this action, explore the parameters of the action through the Qiime2 command line interface: 

In [8]:
! qiime diversity alpha-rarefaction --help

Usage: qiime diversity alpha-rarefaction [OPTIONS]

  Generate interactive alpha rarefaction curves by computing rarefactions
  between `min_depth` and `max_depth`. The number of intermediate depths to
  compute is controlled by the `steps` parameter, with n `iterations` being
  computed at each rarefaction depth. If sample metadata is provided,
  samples may be grouped based on distinct values within a metadata column.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          Feature table to compute rarefaction curves from.
                                                                    [required]
  --i-phylogeny ARTIFACT  Optional phylogeny for phylogenetic metrics.
    Phylogeny[Rooted]                                               [optional]
Parameters:
  --p-max-depth INTEGER   The maximum rarefaction depth. Must be greater than
    Range(1, None)        min-depth.                                [required]
  --p-metrics TEXT... Choices('lladser_pe', 'gin

Now answer a few questions:

**a)** What does the parameter `--p-iterations` control in the action?

* Use the action as described below to calculate the interactive alpha-rarefaction curves:

In [9]:
! qiime diversity alpha-rarefaction \
    --i-table $data_dir/feature-table.qza \
    --i-phylogeny $data_dir/insertion-tree.qza \
    --p-max-depth 10000 \
    --m-metadata-file $data_dir/metadata_proc.tsv \
    --o-visualization $data_dir/alpha-rarefaction.qzv

Saved Visualization to: w7_data/alpha-rarefaction.qzv


In [10]:
Visualization.load(f'{data_dir}/alpha-rarefaction.qzv')

<visualization: Visualization uuid: 7e50f9a8-18e1-4640-b6b2-f2502861011b>

* Investigate the resulting visualization. You will find two plots. The **top plot** displays the selected alpha diversity metrics (e.g. shannon) over different sequencing depths. When the lines in this plot "level out" (slope approximates zero), it indicates that collecting more sequences would not change the estimated samples' diversity metric. 
           
           
**b)** Set the "Sample Metadata Column" to "env" and the "Metric" equal "shannon". At which sampling depth does our shannon diversity metric start to "level out"?
           
      
* Investigate the **bottom plot** in the visualisation. It displays the count of samples that remain per metadata group when the feature table is rarefied to the respective sampling depth (x-axis). It essentially gives you an estimate on how reliable the above represented diversity metrics are (the fewer samples, the more uncertain it is). To select the depth at which to rarefy your samples, you should aim to maximize the rarefying threshold while minimizing loss of samples due to insufficient coverage. 
          
          
**c)** Which sequencing depth would you choose for rarefying these samples?


**d)** With this sequencing depth, how many samples do we end up with? And which "env" group ends up losing most samples? (Hint: Investigate above created "Interactive Sample Detail" tab in feature table summary `w7_data/feature-table.qzv`.)        

<a id='sec3'></a>

# 3. Diversity analysis

Having decided on a sequencing depth for rarefaction, we can now proceed and investigate the within-sample diversity (= **alpha diversity**) as well as between-sample diversity (= **beta diversity**). To do this we will make use of the `q2-diversity` plugin's `core-metrics-phylogenetic` method that rarefies the feature table and calculates a collection of diversity metrics for it. 

* List the outputed diversity metrics of `core-metrics-phylogenetic` through the Q2 command line interface. If a metric is unfamiliar to you, you can consult this [forum post](https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282). 

In [11]:
! qiime diversity core-metrics-phylogenetic --help

Usage: qiime diversity core-metrics-phylogenetic [OPTIONS]

  Applies a collection of diversity metrics (both phylogenetic and non-
  phylogenetic) to a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree containing tip identifiers that
    Phylogeny[Rooted]     correspond to the feature identifiers in the table.
                          This tree can contain tip ids that are not present
                          in the table, but all feature ids in the table must
                          be present in this tree.                  [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                           

* Let's apply this method to our feature table with a rarefaction depth of 1500 sequences per sample:

In [12]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/feature-table.qza \
  --i-phylogeny $data_dir/insertion-tree.qza \
  --m-metadata-file $data_dir/metadata_proc.tsv \
  --p-sampling-depth 1500 \
  --output-dir $data_dir/core-metrics-results

Saved FeatureTable[Frequency] to: w7_data/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: w7_data/core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: w7_data/core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: w7_data/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: w7_data/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: w7_data/core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: w7_data/core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: w7_data/core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: w7_data/core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: w7_data/core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: w7_data/core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: w7_data/core-metrics

<a id='sec3.1'></a>

## 3.1 Alpha diversity

The alpha diversity metric measures the within-sample diversity of a sample or a group of samples. As a continuous value alpha diversities can be tested for significant differences with non-parametric statistical tests:

* Test the associations between categorical metadata columns and the Faith Phylogenetic Diversity (a measure of community richness) metric using the `qiime diversity alpha-group-significance` method  (implementation of a one-way ANOVA method, namely [Kruskal-Wallis test](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_one-way_analysis_of_variance)):

In [13]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/metadata_proc.tsv \
  --o-visualization $data_dir/core-metrics-results/faith-pd-group-significance.qzv

Saved Visualization to: w7_data/core-metrics-results/faith-pd-group-significance.qzv


In [14]:
Visualization.load(f'{data_dir}/core-metrics-results/faith-pd-group-significance.qzv')

<visualization: Visualization uuid: 924976b7-fba4-4bc7-942d-46a9f16b3457>

**a)** Inspect the created visualisation `w7_data/core-metrics-results/faith-pd-group-significance.qzv`. Which categorical sample metadata columns are strongly associated with the differences in microbial community richness? 

* Next, we will test whether numeric sample metadata columns are correlated with microbial community richness by using the `qiime diversity alpha-correlation` method (implementation of [Spearman correlation](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)):

In [15]:
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/metadata_proc.tsv \
  --o-visualization $data_dir/core-metrics-results/faith-pd-group-significance-numeric.qzv

Saved Visualization to: w7_data/core-metrics-results/faith-pd-group-significance-numeric.qzv


In [16]:
Visualization.load(f'{data_dir}/core-metrics-results/faith-pd-group-significance-numeric.qzv')

<visualization: Visualization uuid: db9a7604-27e5-4d06-a593-50d883c08cb1>

**b)** Inspect the created visualisation. Why do some of the test statistics show a p-value of NaN? (Hint: Remember the output of the `df_meta.describe()` method in the exercises of Week2?)
                  
**c)** Does the significant correlation between `host_subject_id` and microbial community richness make sense?
                   

<a id='sec3.2'></a>

## 3.2 Beta diversity

The beta diversity metric measures the between-sample diversity of a sample or a group of samples.        
To inspect groupings of beta diversity metrics across metadata categories, we will inspect the principal coordinates (PCoA) plots created with the `qiime diversity core-metrics-phylogenetic` method before. 
                
**a)** Inspect the `unweighted_unifrac_emperor.qzv` visualisation in Q2 View. According to which category do the beta diversity metrics of the samples cluster?
            


In [17]:
Visualization.load(f'{data_dir}/core-metrics-results/unweighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: a7316015-cf0a-4b3c-ab56-9d00a08b8ece>

Beta diversity metric groupings according to categorical variables can be tested for significant differences with a PERMANOVA. This is a non-parametrics statistical test that checks the null hypothesis that the distances between samples of one group are equivalent to distances to samples of another group. If this null hypothesis is rejected, we can infer that the distances between samples of one group differ significantly from the distances to samples in another group. We can perform a PERMANOVA test checking whether the observed categories are significantly grouped in Q2 with the `qiime diversity beta-group-significance` method: 

In [18]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/metadata_proc.tsv \
    --m-metadata-column env \
    --p-pairwise \
    --o-visualization $data_dir/core-metrics-results/uw_unifrac-env-significance.qzv

Saved Visualization to: w7_data/core-metrics-results/uw_unifrac-env-significance.qzv


In [19]:
Visualization.load(f'{data_dir}/core-metrics-results/uw_unifrac-env-significance.qzv')

<visualization: Visualization uuid: a7a95f2e-1cf8-435c-a75d-943541dc71de>

**b)** Inspect the created visualisation. What does the performed PERMANOVA test tell us about the differences in beta diversity of "env" groupings? What specific pairs of environments are significantly different from each other?           

Inspect another beta diversity metric. What do you observe in the PCoA visualisation? What can you say about the significance of the observed differences?

This is not the only way to run a PERMANOVA test, either. check out the help documentation for the following action, which allows running a multivariate PERMANOVA test. The `adonis` implementation of PERMANOVA (part of the r-vegan package) accepts a formula as input, which can consist of one or more independent terms. See the documentation for more details. **_This might be particularly useful in your group projects for testing which covariates explain the most variation in your datasets._** Enjoy!

In [20]:
! qiime diversity adonis --help

Usage: qiime diversity adonis [OPTIONS]

  Determine whether groups of samples are significantly different from one
  another using the ADONIS permutation-based statistical test in vegan-R.
  The function partitions sums of squares of a multivariate data set, and is
  directly analogous to MANOVA (multivariate analysis of variance). This
  action differs from beta_group_significance in that it accepts R formulae
  to perform multi-way ADONIS tests; beta_group_signficance only performs
  one-way tests. For more details, consult the reference manual available on
  the CRAN vegan page: https://CRAN.R-project.org/package=vegan

Inputs:
  --i-distance-matrix ARTIFACT
    DistanceMatrix     Matrix of distances between pairs of samples.
                                                                    [required]
Parameters:
  --m-metadata-file METADATA...
    (multiple          Sample metadata containing formula terms.
     arguments will    
     be merged)                                 